In [3]:
from pandas import DataFrame, read_excel

In [6]:
dtabarre = DataFrame(
    {
        'case_id':[
            '14172120-aa43-4456-82c3-9c38c5da9aaf',
        ]
    }
)

dtabarre

,case_id
0,14172120-aa43-4456-82c3-9c38c5da9aaf
